In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [16]:
# !pip install langchain-teddynote

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH99-MyTest")

In [9]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings
from tqdm import tqdm
import os
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
# CSV 파일 경로
loader = CSVLoader(
    file_path="./data/announcement.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        "quotechar": '"',  # 인용 부호 문자
        "fieldnames": [
            "Sequence",  # 순번
            "Registration number",  # 공고번호
            "Announcement name",  # 공고명
            "Support areas",  # 지원분야
            "Region",  # 지역
            "Target",  # 지원대상
            "Target age",  # 대상연령
            "Application period",  # 접수기간
            "Entrepreneurial history",  # 업력
            "Institution name",  # 기관명
            "Organization classification",  # 기관구분 : 공공, 민간, 교육
            "Department in charge",  # 담당부서
            "Announcement number",  # 공고 제 호
            "Announcement contents",  # 공고내용
            "Registration date",  # 공고등록일
            "Announcement registrar name",  # 공고 기업명
            "How to apply work-in",  # 신청방법 : 방문
            "How to apply By mail",  # 신청방법 : 우편
            "How to apply By Fax",  # 신청방법 : FAX
            "How to apply By email",  # 신청방법 : email
            "How to apply online",  # 신청방법 : 온라인
            "How to apply other",  # 신청방법 : 기타
            "Who to apply for",  # 신청대상
            "Excluded from application",  # 제외대상
            "Summary",  # 공고명 + 공고내용
        ],  # 필드 이름
    },
)

# 데이터 로드
docs = loader.load()

# 데이터 출력
print(docs[1].page_content)

In [ ]:
print(docs[1])

In [61]:
# XML 변환
xml_docs = []
for doc in docs[1:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>"
    xml_docs.append(row_str)

In [ ]:
print(xml_docs[0])

In [ ]:
# Embedding 함수 설정
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

def reset_collection(client, collection_name):
    try:
        # 기존 컬렉션 존재 여부 확인
        existing_collections = client.list_collections()
        collection_exists = any(
            col.name == collection_name for col in existing_collections
        )

        if collection_exists:
            print(f"기존 컬렉션 '{collection_name}'을 삭제합니다.")
            client.delete_collection(collection_name)
            print(f"컬렉션 '{collection_name}'이 삭제되었습니다.")

        # 새로운 컬렉션 생성
        print(f"새로운 컬렉션 '{collection_name}'을 생성합니다.")
        collection = client.create_collection(
            name=collection_name, embedding_function=embedding_function
        )

        print(f"컬렉션 '{collection_name}'이 성공적으로 초기화되었습니다.")
        return collection

    except Exception as e:
        print(f"컬렉션 초기화 중 오류 발생: {str(e)}")
        raise


# 컬렉션 초기화 실행
collection = reset_collection(client, "my_collection")

# 컬렉션 정보 확인
print("\n=== 초기화된 컬렉션 정보 ===")
print(f"컬렉션 이름: {collection.name}")
print(f"컬렉션 크기: {collection.count()} 문서")

In [69]:
# DataFrame에서 ChromaDB용 데이터 준비 및 저장
def prepare_and_store_data(docs, collection):
    # 배치 크기 설정
    BATCH_SIZE = 100
    total_rows = len(docs)

    idx = 1
    documents = []
    metadatas = []
    ids = []

    # tqdm으로 진행률 표시
    for idx in tqdm(range(total_rows)):
        row = docs[idx]  # .page_content
        documents.append(row)

        ids.append(str(idx))

        # 배치 크기에 도달하면 저장
        if len(documents) == BATCH_SIZE or idx == total_rows - 1:
            print(f"\n배치 저장 중... ({len(documents)} 문서)")
            # collection.add(documents=documents, metadatas=metadatas, ids=ids)
            collection.add(documents=documents, ids=ids)

            # 배치 초기화
            documents = []
            metadatas = []
            ids = []

    return total_rows

In [ ]:
total_documents = prepare_and_store_data(xml_docs, collection)
print(f"\n총 {total_documents}개의 문서가 성공적으로 저장되었습니다.")

In [ ]:
# 저장된 데이터 샘플 확인
print("\n=== 저장된 데이터 샘플 확인 ===")
sample_results = collection.get(ids=["0"], include=["documents", "metadatas"])
print("\nDocument:", sample_results["documents"][0])
print("Metadata:", sample_results["metadatas"][0])

# 컬렉션 정보 출력
print(f"\n현재 컬렉션의 총 문서 수: {collection.count()}")

In [ ]:
# 첫 번째 document 가져오기
result = collection.get(
    ids=["1"],  # 첫 번째 행의 ID
    include=["documents", "embeddings"],  # documents와 embeddings 모두 포함
)

print("Document 내용:")
print(result["documents"][0])  # 첫 번째 document의 텍스트 내용

print("\nEmbedding 벡터 (처음 10개 값):")
print(result["embeddings"][0][:100])  # 임베딩 벡터의 처음 10개 값만 출력

In [82]:
from langchain_openai import ChatOpenAI
import chromadb
from chromadb.utils import embedding_functions
import os

# 쿼리 실행
# query = "서울에 살고있어 내가 지원받을 수 있는 정부지원사업은 뭐가 있지"
# query = "I live in Seoul. What government support projects can I receive support from?"
# query = "창업을 준비 중인데 창업관련 교육을 받고 싶은데 어떤 지원 프로그램이 있나요?"
# query = "사내벤처 프로그램 알려줘"
query = "예비창업자를 지원하는 프로그램은"
# results = collection.query(query_texts=[query], n_results=3)  # 상위 3개 결과 가져오기
results = collection.query(
    query_texts=[query],
    n_results=1,
    # where={
    #     "area" : "서울특별시",
    # }
)

# results["documents"]

In [ ]:
from langchain_chroma import Chroma

persist_db = Chroma(
    persist_directory="./my_db.db",
    embedding_function=embedding_function,
    collection_name="my_collection",
)

# 저장된 데이터 확인
# persist_db.get()

persist_db.similarity_search("예비창업자를 위한 지원 프로그램 알려줘")

In [ ]:
for i, doc in enumerate(results["documents"][0]):
    print(f"\nDocument {i+1}:")
    print(f"Content: {doc}")

In [ ]:
# OpenAI를 사용한 답변 생성
# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

# 프롬프트 구성
prompt = f"""다음은 예비창업자가 지원할 수 있는 지원사업에 대한 검색 결과입니다. 
이를 바탕으로 예비창업자가 지원할 수 있는 주요 지원사업들을 간단명료하게 요약해서 설명해주세요:

{results['documents'][0]}
"""

# ChatGPT API 호출
# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant that explains startup support programs in Korean.",
#         },
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0,
#     max_tokens=1000,
# )
response = llm.invoke(prompt)

print("\n=== AI 답변 ===")
print(response.content)